# HSV - Hue, Saturation, Value Colorspace
Stough, 202-

- [skimage's color module](https://scikit-image.org/docs/dev/api/skimage.color.html)
- [What is HSV?](https://en.wikipedia.org/wiki/HSL_and_HSV)
- [Why HSV?](https://dsp.stackexchange.com/questions/2687/why-do-we-use-the-hsv-colour-space-so-often-in-vision-and-image-processing)

The HSV and closely related HSL colorspaces are ways of considering 'color' that 
separate the brightness of a color from its tone and purity. This is useful because we
as humans are highly atuned to *brightness changes* in our perception. So if the RGB space speaks
to the mechanism by which we receive a color stimulus--activation of red, green, and 
blue-centered cones--then the HSV speaks more to how we *perceive* a color.

<img src="../dip_figs/rgb_hsv_diagram.jpeg" style="width:500px;height:200px"/> <img src="../dip_figs/hsl_cylinder.png" style="width:250px;height:200px"/>

In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
import skimage.color as color
from ipywidgets import VBox, HBox, FloatSlider

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')

import matrix_utils
from vis_utils import (vis_rgb_cube,
                       vis_hsv_cube,
                       vis_hists,
                       lab_uniform)

## Viewing the planes of the HSV space.
Here I'll make a little interactive plot on the HSV space for various saturation, thanks to HYRY's [stackoverflow answer](https://stackoverflow.com/questions/10787103/2d-hsv-color-space-in-matplotlib). We first stack together an HSV image, then convert to RGB so that we can display it.

There is a bit of a problem with these demos. The `plt.ioff()` and `plt.ion()` are required to avoid duplication of plots when putting together an interactive demo. `plt.close()` is used to make sure those plots don't regenerate later in the notebook. 

In [17]:
def changeSat(HSV, newvalue=1.0):
    H, S, V = np.split(HSV, indices_or_sections=3, axis=-1)
    S = newvalue*np.ones_like(V)
    HSV = np.dstack((H,S,V))
    return color.hsv2rgb(HSV)

In [18]:
plt.ioff()
# plt.clf()

# Set up the initial display
V, H = np.mgrid[0:1:100j, 0:1:360j]
S = np.ones_like(V)
HSV = np.dstack((H,S,V))
RGB = color.hsv2rgb(HSV)


# A slider to set the saturation.
sat_slider = FloatSlider(
    orientation='horizontal',
    value=1.0,
    min=0.00,
    max=1.00,
    step=0.01,
    description='$S_{HSV}$'
)

# Setting up the figure.
fig_args = {'num':1, 'frameon':True}
fig, ax = plt.subplots(1,1, figsize=(6,3), **fig_args)
fig.canvas.header_visible = False

thedisp = ax.imshow(RGB, origin='lower', extent=[0, 360, 0, 1], aspect=150)
plt.xlabel('H') # These don't need to change
plt.ylabel('V')
thetext = ax.set_title('$S_{HSV}=1$')
plt.tight_layout()

def update_image(change):
    global thedisp, thetext, HSV
    
    I_rgb = changeSat(HSV, change.new)
    thedisp.set_array(I_rgb)
    thetext.set_text(f'$S_{{HSV}}$={change.new:.02f}')
    fig.canvas.draw()
    fig.canvas.flush_events()

sat_slider.observe(update_image, names='value')

VBox([sat_slider, fig.canvas])


In [4]:
# Close the above interactive demo figure, so that it won't appear later.
plt.close(1)

## Viewing an image as RGB and HSV for comparison

In [5]:
IMAGE = 'mountainSpring.jpg'
# IMAGE = 'peppers.png'
# IMAGE = 'blueridge.jpg'

# Let's construct our own colormaps
cm_KR = mcolors.LinearSegmentedColormap.from_list('kr', [(0,0,0), (1, 0, 0)], 256)
cm_KG = mcolors.LinearSegmentedColormap.from_list('kg', [(0,0,0), (0, 1, 0)], 256)
cm_KB = mcolors.LinearSegmentedColormap.from_list('kb', [(0,0,0), (0, 0, 1)], 256)

In [6]:
Irgb = plt.imread('../dip_pics/' + IMAGE)

# Image of uniform random distributed color
# Irgb = np.random.random((100,100,3))

In [7]:
# plt.clf()
plt.ion()

f, ax = plt.subplots(3,3, figsize = (8,6), sharex=True, sharey=True)

Ihsv = color.rgb2hsv(Irgb)

# Row 0: just the full color image
ax[0][1].imshow(Irgb)
ax[0][0].set_axis_off()
ax[0][2].set_axis_off()

# Row 1: r g b channels
for i in range(3):
    ax[1,i].imshow(Irgb[...,i], cmap=[cm_KR, cm_KG, cm_KB][i])
    ax[1,i].set_title(['Red', 'Green', 'Blue'][i])


# Now to display the HSV space.

cmapNameHue = 'myHue'
colorsH = [(.7,0,0), (.5,.5,0), (0,.7,0), (0,.5,.5), (0,0,.7), (.5, 0, .5), (.7, 0, 0)]
cmHue = mcolors.LinearSegmentedColormap.from_list(cmapNameHue, lab_uniform(colorsH), 256)


ax[2,0].imshow(Ihsv[...,0], cmap=cmHue)
ax[2,0].set_title('H')

ax[2,1].imshow(Ihsv[...,1], cmap='gray')
ax[2,1].set_title('S')

ax[2,2].imshow(Ihsv[...,2], cmap='gray')
ax[2,2].set_title('V')

plt.tight_layout()
f.canvas.set_window_title('RGB vs HSV')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
matrix_utils.arr_info(Ihsv)

((1200, 1600, 3), dtype('float64'), 0.0, 1.0)

&nbsp;

## Let's visualize in 3D

Given the HSV graph above, Hue can be seen as P, and Saturation as R in polar coordinates. Value 
is the Z. Easy. So then, how to [plot in polar coords](https://matplotlib.org/3.1.0/gallery/mplot3d/surface3d_radial.html)?

In [9]:
vis_hsv_cube(Irgb)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

&nbsp;

## Manipulating the HSV components of an image...
In the below interactive visualization we can manipulate the hue of an image. Try to complete the application by adding saturation and value sliders to this image manipulation tool.

Thanks XKCD:

<a href="https://xkcd.com/648/"><img src="https://imgs.xkcd.com/comics/fall_foliage.png" style="height:300px"/></a>


In [88]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
import skimage.color as color
from ipywidgets import VBox, HBox, FloatSlider

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')

import matrix_utils
from vis_utils import (vis_rgb_cube,
                       vis_hsv_cube,
                       vis_hists,
                       lab_uniform)


# I like this image more 
Irgb = plt.imread('../dip_pics/blueridge.jpg')
Ihsv = color.rgb2hsv(Irgb)

In [89]:
def changeHue(newhue):
    global Ihsv 
    change = newhue - Ihsv[...,0].mean()
    
    Ic = Ihsv.copy()
    Ic[...,0] = np.fmod(Ic[...,0]+change, 1) # anything > 1 wraps around 0.
    
    return np.clip(Ic, 0, 1)

In [90]:
plt.ioff()
# plt.clf()

hue_mean = Ihsv[...,0].mean()

hue_slider = FloatSlider(
    orientation='horizontal',
    value=hue_mean,
    min=0.00,
    max=1.00,
    step=0.01,
    description='Hue'
)

fig_args = {'num':' ', 'frameon':True, 'sharex':True, 'sharey':True}
fig, ax = plt.subplots(1,2, figsize=(8,3), **fig_args) 

I_h = changeHue(hue_mean)
I_rgb = color.hsv2rgb(I_h)

# display artists I'll update
adisp = ax[0].imshow(Irgb)
rdisp = ax[1].imshow(I_rgb)

ax[0].set_title(f'Original ({hue_mean:.03f})')
rtext = ax[1].set_title(f'Hue: {hue_mean:.03f}')


def update_image(change):
    global Ihsv, I_h, adisp, rdisp, rtext
    
    I_rgb = color.hsv2rgb(changeHue(change.new)) 
    rdisp.set_array(I_rgb)
    rtext.set_text(f'Hue: {change.new:.03f}') #hue_slider.value
    fig.canvas.draw()
    fig.canvas.flush_events()

hue_slider.observe(update_image, names='value')

plt.tight_layout()

VBox([hue_slider, fig.canvas])

In [97]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
import skimage.color as color
from ipywidgets import VBox, HBox, FloatSlider

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')

import matrix_utils
from vis_utils import (vis_rgb_cube,
                       vis_hsv_cube,
                       vis_hists,
                       lab_uniform)


# I like this image more 
Irgb = plt.imread('../dip_pics/blueridge.jpg')
Ihsv = color.rgb2hsv(Irgb)

In [98]:
def changeHue(newhue):
    global Ihsv
    change = newhue - Ihsv[...,0].mean()
    
    Ic = Ihsv.copy()
    Ic[...,0] = np.fmod(Ic[...,0]+change, 1) # anything > 1 wraps around 0.
    
    return np.clip(Ic, 0, 1)


def changeSat(newSat):
    global Ihsv
    #change = newSat - Ihsv[...,0].mean()
    
    #H, S, V = np.split(Ihsv, indices_or_sections=3, axis=-1)
    #S = newSat*np.ones_like(V)
    #Ihsv = np.dstack((H,S,V))
    #Ic = Ihsv.copy()
    
    #Ic[...,0] = np.fmod(Ic[...,0]+change, 1) # anything > 1 wraps around 0.
    
    
    Ihsv = color.rgb2hsv(Irgb)
    Ihsv[...,1]=np.where(Ihsv[...,1]*newSat > 1.0, 1.0, Ihsv[...,1]*newSat)
    
    return np.clip(Ihsv, 0, 1)
                   

def changeVal(newVal):
    global Ihsv
    
    change = newVal - Ihsv[...,0].mean()
    
    Ic = Ihsv.copy()
    Ic[...,0] = np.fmod(Ic[...,0]+change, 1)
    
    return np.clip(Ic, 0, 1)



In [99]:
plt.ioff()
# plt.clf()
hue_mean = Ihsv[...,0].mean()

sat_mean = .5

val_mean = .5


hue_slider = FloatSlider(
    orientation='horizontal',
    value=hue_mean,
    min=0.00,
    max=1.00,
    step=0.01,
    description='Hue'
)

sat_slider = FloatSlider(
    orientation='horizontal',
    value=0.5,
    min=0.00,
    max=1.00,
    step=0.01,
    description='Sat'
)

val_slider = FloatSlider(
    orientation='horizontal',
    value=.5,
    min=0.00,
    max=1.00,
    step=0.01,
    description='Val'
)



fig_args = {'num':' ', 'frameon':True, 'sharex':True, 'sharey':True}
fig, ax = plt.subplots(1,2, figsize=(8,3), **fig_args) 

I_h = changeHue(hue_mean)
I_s = changeSat(0.5)
I_v = changeVal(.5)
IH_rgb = color.hsv2rgb(I_h)
IS_rgb = color.hsv2rgb(I_s)
IV_rgb = color.hsv2rgb(I_v)


# display artists I'll update
adisp = ax[0].imshow(Irgb)
rdisp = ax[1].imshow(IH_rgb)

ax[0].set_title(f'Original')
rtext = ax[1].set_title(f'H: {hue_mean:.03f}' + \' S: {sat_mean:.03f}'' V: {val_mean:.03f}')


def update_image(change):
    global Ihsv, I_h, I_s, I_v, adisp, rdisp, rtext, IH_rgb, IS_rgb, IV_rgb
    
    #Change Hue
    IH_rgb = color.hsv2rgb(changeHue(change.new))
    
    rdisp.set_array(IH_rgb)
    rtext.set_text(f'H: {change.new:.03f}') #hue_slider.value
    fig.canvas.draw()
    fig.canvas.flush_events()
    
    #Change Saturation
    IS_rgb = color.hsv2rgb(changeSat(change.new))
    
    rdisp.set_array(IS_rgb)
    rtext.set_text(f'S: {change.new:.03f}') #saturation_slider.value
    fig.canvas.draw()
    fig.canvas.flush_events()
    
    #Change Value
    IV_rgb = color.hsv2rgb(changeVal(change.new))
    
    rdisp.set_array(IV_rgb)
    rtext.set_text(f'V: {change.new:.03f}') #value_slider.value
    fig.canvas.draw()
    fig.canvas.flush_events()

hue_slider.observe(update_image, names='value')
sat_slider.observe(update_image, names='value')
val_slider.observe(update_image, names='value')

plt.tight_layout()

VBox([HBox([hue_slider, sat_slider, val_slider]), fig.canvas])

In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
import skimage.color as color
from ipywidgets import VBox, HBox, FloatSlider

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')

import matrix_utils
from vis_utils import (vis_rgb_cube,
                       vis_hsv_cube,
                       vis_hists,
                       lab_uniform)


# I like this image more 
Irgb = plt.imread('../dip_pics/blueridge.jpg')
Ihsv = color.rgb2hsv(Irgb)

In [2]:
def changeHSV(new):
    global Irgb, hue_slider, sat_slider, val_slider, I_change, Ihsv
    HSV = color.rgb2hsv(Irgb)

    H, S, V = np.split(HSV, indices_or_sections=3, axis=-1)
    
    H2 = H.copy()
    H2 = (1-new)*H.min() + new*H.max()
    H2 = H2.clip(0, 1)
    
    
    S2 = S.copy()
    S2 = (1-new)*S.min() + new*S.max()
    #S2[S2 > .2] = S2[S2 > .2] + new
    S2 = S2.clip(0, 1)
    
    V2 = V.copy()
    V2 = (1-new)*V.min() + new*V.max()
    V2 = V2.clip(0, 1)
    
    HSV = np.dstack((H2,S2,V2))
    I_change = HSV.copy()
    return I_change


In [3]:
plt.ioff()
# plt.clf()

hue_mean = Ihsv[...,0].mean()

sat_mean = Ihsv[...,1].mean()

val_mean = Ihsv[...,2].mean()


hue_slider = FloatSlider(
    orientation='horizontal',
    value=hue_mean,
    min=0.00,
    max=1.00,
    step=0.01,
    description='Hue'
)

sat_slider = FloatSlider(
    orientation='horizontal',
    value=sat_mean,
    min=0.00,
    max=1.00,
    step=0.01,
    description='Sat'
)

val_slider = FloatSlider(
    orientation='horizontal',
    value=val_mean,
    min=0.00,
    max=1.00,
    step=0.01,
    description='Val'
)



fig_args = {'num':' ', 'frameon':True, 'sharex':True, 'sharey':True}
fig, ax = plt.subplots(1,2, figsize=(8,3), **fig_args) 

# display artists I'll update
adisp = ax[0].imshow(Irgb)
rdisp = ax[1].imshow(Ihsv)

ax[0].set_title(f'Original')
rtext = ax[1].set_title(f'H: {hue_slider.value:.03f}' + f'S: {sat_slider.value:.03f}' + f'V: {val_slider.value:.03f}')

def update_image(change):
    global Ihsv, adisp, rdisp, rtext, I_change, hue_slider, val_slider, sat_slider, Irgb
    
    IH_rgb = (changeHSV(hue_slider.value))

    IS_rgb = (changeHSV(sat_slider.value))

    IV_rgb = (changeHSV(val_slider.value))

    I_change = color.hsv2rgb(I_change)
    
    rdisp.set_array(I_change)
    
    rtext.set_text(f'H: {hue_slider.value:.03f}' + f'S: {sat_slider.value:.03f}' + f'V: {val_slider.value:.03f}')      
    fig.canvas.draw()
    fig.canvas.flush_events()
    

hue_slider.observe(update_image, names='value')
sat_slider.observe(update_image, names='value')
val_slider.observe(update_image, names='value')

plt.tight_layout()

VBox([HBox([hue_slider, sat_slider, val_slider]), fig.canvas])

In [16]:



fig, ax = plt.subplots(3,1, figsize = (8,8))
fig.subplots_adjust(hspace = .4)

a = image_hsv[:,:,0]
hist, bins = np.histogram(a, bins=100, normed=True)
bin_centers = (bins[1:]+bins[:-1])*0.5
ax[0].plot(bin_centers, hist)
ax[0].set(title = 'Hue')

a = image_hsv[:,:,1]
hist, bins = np.histogram(a, bins=100, normed=True)
bin_centers = (bins[1:]+bins[:-1])*0.5
ax[1].plot(bin_centers, hist)
ax[1].set(title = 'Saturation', xlim = [0,1])

a = image_hsv[:,:,2]
hist, bins = np.histogram(a, bins=100, normed=True)
bin_centers = (bins[1:]+bins[:-1])*0.5
ax[2].plot(bin_centers, hist)
ax[2].set(title = 'Value', xlim = [0,1])

NameError: name 'image_hsv' is not defined